In [2]:
library(data.table)
library(dplyr)
library(BigVAR)
library(forecast)
library(DescTools)

In [3]:
file_donorA = '/storage/zkarwowska/microbiome-interactions/datasets/processed/ready_datasets_transformed/common/donorA.csv'
file_donorB = '/storage/zkarwowska/microbiome-interactions/datasets/processed/ready_datasets_transformed/common/donorB.csv'
file_M = '/storage/zkarwowska/microbiome-interactions/datasets/processed/ready_datasets_transformed/common/male.csv'
file_F = '/storage/zkarwowska/microbiome-interactions/datasets/processed/ready_datasets_transformed/common/female.csv'

# MODEL

In [29]:
construct_interaction_matrix<-function(file, title){
    
    #' Fit a dataframe with all timesteps as samples and
    #' top 50 bacteria to a BigVar model with lag 1.
    #' It returns an interactions matrix
    
    #read file
    df <- read.table(file, sep = ",", header = TRUE, row.names="X")
    #get top 50 bacteria
    df_top50 <- df[,1:50]

    #scale
    x_scaled = scale(df_top50)
    #x_test_scaled = scale(x_test, center=attr(x_train_scaled, "scaled:center"), #if split into train and test
                                  #scale=attr(x_train_scaled, "scaled:scale"))
    # difference
    x_scaled_diff = as.matrix(diff(x_scaled, differences = 1, lag = 1))
    x_scaled_diff_matrix = as.matrix(x_scaled_diff)

    #fit a BigVAR model
    p = 1    #lag
    m1=constructModel(x_scaled_diff_matrix,
                      p=p,
                      struct="Basic", #lasso penalty
                      gran=c(150,10),
                      verbose=FALSE,
                      IC=TRUE                    
                     )
    #cross validate to find penalty params
    results=cv.BigVAR(m1)
    #get coefficients matrix
    B2=results@betaPred

    #change matrix into a dataframe and save
    COEF_MATRIX_DF <- as.data.frame(B2)
    COEF_MATRIX_DF['otu'] <- colnames(df_top50)
    write.csv(COEF_MATRIX_DF, title)
}

In [34]:
construct_interaction_matrix(file_F, '/storage/zkarwowska/microbiome-interactions/datasets/processed/female_interaction_m.csv')